In [1]:
import plotly
import pandas
import numpy
import scipy
import plotly.plotly as py
import plotly.graph_objs as go

from plotly.graph_objs.scatter import Marker

OUR_MAX_FITNESS = {
    "warmup": 268,
    "anchors": 355,
    "ranges": 212,
    "backrefs": 423,
    "abba": 439,
    "aman-aplan": 302,
    "prime": 1035,
    "four": 471,
    "order": 277,
    "triples": 378,
    "glob": 1255,
    "balance": 2209,
    "powers": 4396,
    "long-count": 1659,
    "alphabetical": 1632,
}

# Fitness máximo possível para cada instância do problema
MAX_FITNESS = {
    "warmup": 21,
    "anchors": 21,
    "ranges": 21,
    "backrefs": 21,
    "abba": 22,
    "aman-aplan": 19,
    "prime": 20,
    "four": 21,
    "order": 21,
    "triples": 21,
    "glob": 21,
    "balance": 32,
    "powers": 11,
    "long-count": 1,
    "alphabetical": 17,
}

# Fitness que o Bartoli conseguiu segundo expressão regular gerada
BARTOLI_FITNESS = {
    "warmup": 21,
    "anchors": 21,
    "ranges": 21,
    "backrefs": 18,
    "abba": 20,
    "aman-aplan": 16,
    "prime": 13,
    "four": 21,
    "order": 21,
    "triples": 15,
    "glob": 19,
    "balance": 29,
    "powers": 7,
    "long-count": 1,
    "alphabetical": 12,
}

NORVIG_FITNESS = {
    "warmup": 21,
    "anchors": 21,
    "ranges": 21,
    "backrefs": 21,
    "abba": 21,
    "aman-aplan": 19,
    "prime": 19,
    "four": 21,
    "order": 21,
    "triples": 21,
    "glob": 0,
    "balance": 31,
    "powers": 11,
    "long-count": 1,
    "alphabetical": 17,
}


COLOR_GREEN = "#00b100"
COLOR_RED   = "#cc0000"
COLOR_BLUE  = "#0000cc"
COLOR_PURPLE = "#663399"
COLOR_ORANGE = "#ff4500"
COLOR_GRAY   = "#827689"



COLORS = ["#0dba86", "#0d98ba", "#0d42ba"]


OUR_FITNESS_LINE = go.Scatter(
    name="Pontuação Máxima",
    x=list(OUR_MAX_FITNESS.keys()),
    y=list(OUR_MAX_FITNESS.values()),
    mode='lines',
)

FITNESS_LINE = go.Scatter(
    name="Pontuação Máxima",
    x=list(MAX_FITNESS.keys()),
    y=list(MAX_FITNESS.values()),
    mode='markers',
    marker=dict(color=COLOR_GRAY, symbol="triangle-up"),
)


BARTOLI_LINE = go.Scatter(
    name="Genético (Bartoli)",
    x=list(BARTOLI_FITNESS.keys()),
    y=list(BARTOLI_FITNESS.values()),
    mode='markers',
    marker=dict(color=COLOR_RED, symbol="square"),
)

NORVIG_LINE = go.Scatter(
    name="Exato (Norvig)",
    x=list(NORVIG_FITNESS.keys()),
    y=list(NORVIG_FITNESS.values()),
    mode='markers',
    marker=dict(color=COLOR_ORANGE, symbol="diamond"),
)



# Importando os dados
ILS_S = pandas.read_csv("./depth_2/ILS_Shrink.csv") #.sort_values(by=['Nome'])

DEPTHS = pandas.unique(ILS_S.Depth)
DEPTHS.sort()

depths = {}
for i in DEPTHS:
    depths[i] = ILS_S[ILS_S.Depth == i]


# Número de timeouts

def get_timeout_graph(title, d):
    nomes = pandas.unique(d.Nome)
    timedout = []
    not_timedout = []
    
    for nome in pandas.unique(d.Nome):
            
        
        i = len(d[(d.Nome == nome) & (d.Timed_out == 0)].Nome)
        not_timedout.append(i)

        i = len(d[(d.Nome == nome) & (d.Timed_out == 1)].Nome)
        timedout.append(i)
        
    trace1 = go.Bar(
        name='ok', 
        x=nomes,
        y=not_timedout,
        marker=dict(color=COLOR_GREEN),
        textposition='auto',
        text=not_timedout)
    
    trace2 = go.Bar(
        name='Timedout',
        x=nomes,
        y=timedout,
        marker=dict(color=COLOR_RED),
        textposition='auto',
        text=timedout)
    
    layout = go.Layout(title=title, barmode='stack')
    fig = go.Figure(data=[trace1, trace2], layout=layout)
    return py.iplot(fig, filename=title)

In [2]:
get_timeout_graph('ILS_S 16: ok / Timedout', ILS_S)

In [3]:
ONE_MINUTE_LINE = go.Scatter(name="1 minute",
                             x=ILS_S.Nome,
                             y=[60 for i in ILS_S.Nome],
                             mode='lines')

TWO_MINUTES_LINE = go.Scatter(name="2 minutes",
                              x=ILS_S.Nome,
                              y=[120 for i in ILS_S.Nome],
                              mode='lines')


data = [go.Box(name="Depth %s" % i, x=depth.Nome, y=depth.Tempo_total) for i, depth in depths.items()]
data.append(ONE_MINUTE_LINE)
# data.append(TWO_MINUTES_LINE)

layout = go.Layout(title="Tempo total de execução em seconds",
                xaxis=dict(title='Depth'),
                yaxis=dict(title='Tempo Total'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)


In [4]:
# Médias de número de comparações para encontrar melhor fitness
MAX_LINE = go.Scatter(name="500k line", x=ILS_S.Nome.unique(), y=[500000 for i in range(15)], mode='lines', marker=Marker(color='rgb(77,77,255)'))


data = [go.Box(name="Depth %s" % i, x=depth.Nome, y=depth.Numero_de_comparacoes) for i, depth in depths.items()]
data.append(MAX_LINE)


layout = go.Layout(title="Número de comparações até encontrar Melhor Fitness",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='Quantidade de comprações'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [5]:
data = [go.Box(name="Depth %s" % i, x=depth.Nome, y=depth.Tempo_para_encontrar_melhor_solucao) for i, depth in depths.items()]
data.append(ONE_MINUTE_LINE)
data.append(TWO_MINUTES_LINE)

layout = go.Layout(title="Tempo até encontrar a melhor solução (em segundos)",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='Tempo até melhor solução'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [6]:
# Boxplot comparando 'melhor fitness'

data = [go.Box(name="Depth %s" % i, x=depth.Nome, y=depth.Melhor_fitness) for i, depth in depths.items()]
data.extend([FITNESS_LINE, BARTOLI_LINE, NORVIG_LINE])


layout = go.Layout(title="Melhor fitness",
                xaxis=dict(title='Instâncias'),
                yaxis=dict(title='fitness'),
                boxmode='group')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)
